In [1]:
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import pandas as pd
from torch import tensor
import numpy as np
from torch.utils.data import Dataset
from sklearn.metrics import mean_squared_error
import random
import os
import matplotlib.pyplot as plt
#from numba import jit
import pickle
from scipy.interpolate import interp1d
from torch.utils.data import DataLoader, random_split
import torch
from torchsummary import summary
import seaborn as sns
import sys
import torch.nn.functional as F
import pywt
from sklearn.preprocessing import MinMaxScaler
from torch.cuda import FloatTensor

# Req for package
sys.path.append("../")
from SkinLearning.Utils.NN import train, test, DEVICE


torch.backends.cudnn.benchmark = True

In [2]:
seed = 123
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

# Wavelet decomposition

In [3]:
def waveletExtraction(signals, wavelet='db4', level=2):
    """
    Extract wavelet packet features for each signal in the input array.

    Args:
    - signals (numpy.ndarray): a 2D array containing two 1D signals
    - wavelet (str): the name of the wavelet to use for decomposition
    - level (int): the level of wavelet packet decomposition to use

    Returns:
    - features (dict): a dictionary containing the wavelet packet features for each signal
    """

    # Initialize dictionary to store features for each signal
    features = []

    # Loop over each signal
    for i in range(2):
        # Perform wavelet packet decomposition
        coeffs = pywt.WaveletPacket(data=signals[i], wavelet=wavelet, mode='symmetric', maxlevel=level).get_level(level, 'freq')

        # Vectorize the coefficients
        coefficients = np.array([c.data for c in coeffs], dtype=object)
        coefficients = coefficients.reshape(-1)

        # Store the coefficients as features for this signal
        features.append(coefficients.tolist())

    return features


In [4]:
waveletExtraction(dataset[0]['input'].cpu().numpy())

NameError: name 'dataset' is not defined

# Dataset

In [16]:
# Folder name will correspond to index of sample
class SkinDataset(Dataset):
    def __init__(self, scaler, signalFolder="D:/SamplingResults2", sampleFile="../Data/newSamples.pkl", runs=range(65535), steps=128):
        # Load both disp1 and disp2 from each folder
        # Folders ordered according to index of sample
        # Use the corresponding sample as y -> append probe?
        self.input = []
        self.output = []
        
        with open(f"{sampleFile}", "rb") as f:
             samples = pickle.load(f)
        
        self.min = np.min(samples[runs])
        self.max = np.max(samples[runs])
        
        
        for run in tqdm(runs):
            inp = []
            fail = False
            
            files = os.listdir(f"{signalFolder}/{run}/")
            
            if files != ['Disp1.csv', 'Disp2.csv']:
                continue
            
            for file in files:
                a = pd.read_csv(f"{signalFolder}/{run}/{file}")
                a.rename(columns = {'0':'x', '0.1': 'y'}, inplace = True)
                
                # Skip if unconverged
                if a['x'].max() != 7.0:
                    fail = True
                    break

                # Interpolate curve for consistent x values
                xNew = np.linspace(0, 7, num=steps, endpoint=False)
                interped = interp1d(a['x'], a['y'], kind='cubic', fill_value="extrapolate")(xNew)
                    
                
                inp.append(interped.astype("float32"))
            
            if not fail:
                if len(inp) != 2:
                    raise Exception("sdf")

                self.input.append(inp)
                self.output.append(samples[int(run)])
        
        scaler.fit(self.output)
        self.output = scaler.fit_transform(self.output)
        self.output = tensor(self.output).type(FloatTensor)
        
        self.input = [waveletExtraction(sample) for sample in self.input]
        self.input = tensor(self.input).type(FloatTensor)
        
        
    def __len__(self):
        return len(self.output)
    
    def __getitem__(self, idx):
        sample = {"input": self.input[idx], "output": self.output[idx]}
        return sample
    
    

In [17]:
"""
    Creates the data set from filtered samples
    Returns the dataset and the scaler
"""
def getDataset(**kwargs):
    # Get filtered data
    if not 'runs' in kwargs.keys():
        with open("../Data/filtered.pkl", "rb") as f:
            runs = pickle.load(f)

        kwargs['runs'] = runs

    scaler = MinMaxScaler()
    dataset = SkinDataset(scaler=scaler, **kwargs)

    return dataset, scaler

In [18]:
"""
    Creates a train/test split from the given data
    Returns train and test data loaders
"""
def getSplit(dataset, p1=0.8):
    train_n = int(p1 * len(dataset))
    test_n = len(dataset) - train_n
    train_set, test_set = random_split(dataset, [train_n, test_n])

    return DataLoader(train_set, batch_size=32, shuffle=True), \
        DataLoader(test_set, batch_size=32, shuffle=True)

In [19]:
dataset, scaler = getDataset()

100%|██████████████████████████████████████████████████████████████████████████████| 2241/2241 [00:28<00:00, 78.71it/s]


In [20]:
train_loader, test_loader = getSplit(dataset)

In [10]:
len(dataset[0]['input'][0])

512

# Model

In [26]:
class RNN(nn.Module):
    def __init__(self):
        super(RNN, self).__init__()
        self.conv1 = nn.Conv1d(2, 128, kernel_size=5, padding=1, bias=False)
        self.pool1 = nn.MaxPool1d(kernel_size=5, stride=2)
        self.bn1 = nn.BatchNorm1d(128)
        
        self.conv2 = nn.Conv1d(128, 256, kernel_size=3, padding=1, bias=False)
        self.pool2 = nn.MaxPool1d(kernel_size=2, stride=2)
        self.bn2 = nn.BatchNorm1d(256)
        
        
        self.rnn = nn.RNN(126, 256, batch_first=True)
        self.fc1 = nn.Linear(65536, 1024)
        self.d1 = nn.Dropout(0.5)
        
        self.fc2 = nn.Linear(1024 , 512)
        self.d2 = nn.Dropout(0.5)
        
        self.fc3 = nn.Linear(512, 128)
        self.d3 = nn.Dropout(0.5)
        
        self.fc4 = nn.Linear(128, 6)

    def forward(self, x):
        batch_size = x.shape[0]
        x = self.pool1(torch.relu(self.bn1(self.conv1(x))))
        x = self.pool2(torch.relu(self.bn2(self.conv2(x))))
        
        h0 = torch.zeros(1, batch_size, 256).to(x.device)
        x, _ = self.rnn(x, h0)
        x = x.reshape(batch_size, -1)
        x = self.d1(torch.relu(self.fc1(x)))
        
        x = self.d2(torch.relu(self.fc2(x)))
        
        x = self.d3(torch.relu(self.fc3(x)))
        x = self.fc4(x)
        
        x = x.view(batch_size, 6)
        return x

In [12]:
class LSTM(nn.Module):
    def __init__(self, input_dim=2, hidden_dim=256, num_layers=1, output_dim=6):
        super(LSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        
        self.fc = nn.Sequential(
            nn.Linear(hidden_dim, 1024),
            nn.ReLU(),
            nn.Linear(1024, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(64, 6)
        
        )

    def forward(self, x):
        batch_size = x.shape[0]
        x = x.reshape(batch_size, -1, 2)
        # Initialize hidden state with zeros
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()
        # Initialize cell state with zeros
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()
        # Move tensors to GPU if available
        if torch.cuda.is_available():
            h0 = h0.cuda()
            c0 = c0.cuda()
            x = x.cuda()
        # Forward propagate LSTM
        out, _ = self.lstm(x)
        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        return out


In [54]:
class RNNSingle(nn.Module):
    def __init__(self, input_dim=2, hidden_dim=256, num_layers=2, output_dim=6):
        super(RNNSingle, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_dim, hidden_dim, num_layers, batch_first=True)
        
        self.fc = nn.Sequential(
            nn.Linear(131072, 1024),
            nn.Dropout(0.5),
            nn.ReLU(),
            nn.Linear(1024, 256),
            nn.Dropout(0.5),
            nn.ReLU(),
            nn.Linear(256, 64),
            nn.Dropout(0.5),
            nn.ReLU(),
            nn.Linear(64, 6)
        
        )


    def forward(self, x):
        batch_size = x.shape[0]
        x = x.reshape(batch_size, -1, 2)
        
        h0 = torch.zeros(5, batch_size, 256).to(x.device)
        x, _ = self.rnn(x, h0)
        x = x.reshape(batch_size, -1)
        
        x = self.fc(x)
        
        return x


In [66]:
class RNN2(nn.Module):
    def __init__(self, input_size=2, hidden_size=256, output_size=6):
        super(RNN2, self).__init__()
        
        # Define the RNN layers for each input channel
        self.rnn1 = nn.RNN(8, 128, batch_first=True)
        self.rnn2 = nn.RNN(8, 128, batch_first=True)
        
        # Define the final RNN layer that takes in the concatenated outputs of the two channels
        self.final_rnn = nn.RNN(2, hidden_size, batch_first=True)
        
        # Define the output layer that maps from the hidden state to the output
        
        self.fc = nn.Sequential(
            nn.Linear(hidden_size, 1024),
            nn.ReLU(),
            nn.Linear(1024, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(64, 6)
        
        )
    
    def forward(self, x):
        batch_size = x.shape[0]
        x = x.reshape(batch_size, 2, 22, 8)
        # Process the two input channels through their respective RNN layers
        _, h1 = self.rnn1(x[:, 0, :, :])
        _, h2 = self.rnn2(x[:, 1, :, :])
        
        
        # Concatenate the hidden states of the two channels
        h_cat = torch.cat([h1, h2])
        h_cat = h_cat.reshape(batch_size, -1, 2)

        # Pass the concatenated hidden states through the final RNN layer
        _, h_final = self.final_rnn(h_cat)
        
        # Compute the output
        output = self.fc(h_final).reshape(batch_size, 6)
        
        return output

In [265]:
class SiameseRNN(nn.Module):
    def __init__(self, input_size=148, hidden_size=128):
        super(SiameseRNN, self).__init__()
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size=input_size, hidden_size=hidden_size, num_layers=1, batch_first=True)
        self.fc = nn.Sequential(
            nn.Linear(hidden_size * 2, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 6),
        )
    
    def forward(self, x):
        batch_size = x.shape[0]
        x1 = x[:, 0, :].unsqueeze(1)
        x2 = x[:, 0, :].unsqueeze(1)
        
        h0 = torch.zeros(1, batch_size, self.hidden_size).to(DEVICE)
        _, h1 = self.rnn(x1, h0)  # Add a batch dimension
        _, h2 = self.rnn(x2, h0)  # Add a batch dimension
        
        out = torch.cat([h1[-1], h2[-1]], dim=1)
        out = out.reshape(batch_size, -1)
        out = self.fc(out)
        return out

In [247]:
class SiameseLSTM(nn.Module):
    def __init__(self, input_size=148, hidden_size=80, num_layers=2):
        super(SiameseLSTM, self).__init__()
        
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        
        # Define the LSTM layer
        self.lstm = nn.LSTM(input_size=input_size, 
                            hidden_size=hidden_size, 
                            num_layers=num_layers, 
                            batch_first=True)
        
        self.fc = nn.Sequential(
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 6),
        )

     
    def forward(self, x):
        batch_size = x.shape[0]
        x1 = x[:, 0, :].unsqueeze(1)
        x2 = x[:, 0, :].unsqueeze(1)
        
        
        h0 = torch.zeros(self.num_layers*1, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers*1, x.size(0), self.hidden_size).to(x.device)
        
        # Forward pass through the LSTM layer
        out1 = self.lstm(x1, (h0, c0))[1][-1]
        out2 = self.lstm(x2, (h0, c0))[1][-1]
        
        
        out = torch.cat([out1, out2], dim=0)
        out = out.reshape(batch_size, -1)
        # Pass the last hidden state through the fully connected layer
        out = self.fc(out)
        
        return out

# Test on all samples

In [266]:
sRNN = SiameseRNN()

In [267]:
sRNN_train_loss, sRNN_val_loss =  train(train_loader, sRNN, val_loader=test_loader, LR=0.01, epochs=400)

Using: cuda:0


Epoch 29/400:  36%|███████████▊                     | 20/56 [00:00<00:00, 170.21batch/s, lastLoss=0.157, valLoss=0.152]


KeyboardInterrupt: 

In [236]:
sLSTM = SiameseLSTM()

In [237]:
sLSTM_train_loss, sLSTM_val_loss =  train(train_loader, sLSTM, val_loader=test_loader, LR=0.01, epochs=400)

Using: cuda:0


Epoch 136/400: 100%|████████████████████████████████| 56/56 [00:00<00:00, 164.01batch/s, lastLoss=0.185, valLoss=0.185]


Epoch 272/400: 100%|████████████████████████████████| 56/56 [00:00<00:00, 161.19batch/s, lastLoss=0.185, valLoss=0.188]


Epoch 400/400: 100%|████████████████████████████████| 56/56 [00:00<00:00, 160.45batch/s, lastLoss=0.185, valLoss=0.192]


Average train loss: 0.18557937770682786
Average validation loss: 0.1850056944147994


In [250]:
test(test_loader, sRNN, scaler)

  0%|                                                                                       | 0/15 [00:00<?, ? batch/s]


ValueError: Found array with dim 3. Estimator expected <= 2.

In [67]:
rnn2 = RNN2()

In [101]:
train_loss2, val_loss =  train(train_loader, rnn2, val_loader=test_loader, LR=0.0001, epochs=400)

NameError: name 'rnn2' is not defined

In [171]:
rnn = RNN()

In [172]:
train_loss, val_loss =  train(train_loader, rnn, val_loader=test_loader, LR=0.0001, epochs=400)

Using: cuda:0


Epoch 94/400:  14%|████▊                             | 8/56 [00:00<00:01, 35.32batch/s, lastLoss=0.099, valLoss=0.0725]


KeyboardInterrupt: 

In [16]:
lstm = LSTM()

In [17]:
lstm_train_loss, lstm_val_loss =  train(train_loader, lstm, val_loader=test_loader, LR=0.0001, epochs=400)

Using: cuda:0


Epoch 136/400: 100%|█████████████████████████████████| 56/56 [00:01<00:00, 42.18batch/s, lastLoss=0.187, valLoss=0.183]


Epoch 272/400: 100%|█████████████████████████████████| 56/56 [00:01<00:00, 39.71batch/s, lastLoss=0.185, valLoss=0.184]


Epoch 400/400: 100%|█████████████████████████████████| 56/56 [00:01<00:00, 45.86batch/s, lastLoss=0.186, valLoss=0.183]


Average train loss: 0.18793364834173448
Average validation loss: 0.18375265919479233


In [55]:
rnnSingle = RNNSingle()

In [56]:
rnnSingle_train_loss, rnnSingle_val_loss =  train(train_loader, rnnSingle, val_loader=test_loader, LR=0.0001, epochs=400)

Using: cuda:0


Epoch 136/400: 100%|█████████████████████████████████| 56/56 [00:04<00:00, 13.24batch/s, lastLoss=0.122, valLoss=0.113]


Epoch 272/400: 100%|█████████████████████████████████| 56/56 [00:04<00:00, 13.16batch/s, lastLoss=0.116, valLoss=0.109]


Epoch 365/400: 100%|█████████████████████████████████| 56/56 [00:04<00:00, 12.94batch/s, lastLoss=0.114, valLoss=0.111]


KeyboardInterrupt: 

Using the raw coefficients doesnt seem to be effective at any level of decomposition:
- The CNN + RNN performs slightly worse with the wavelet packet decomposition
- Using some levels
- Found an RNN alone that was able to perform a little worse than CNN + RNN

# Try not flattenning coefficients

In [268]:
def waveletExtraction(signals, wavelet='db4', level=3):
    """
    Extract wavelet packet features for each signal in the input array.

    Args:
    - signals (numpy.ndarray): a 2D array containing two 1D signals
    - wavelet (str): the name of the wavelet to use for decomposition
    - level (int): the level of wavelet packet decomposition to use

    Returns:
    - features (dict): a dictionary containing the wavelet packet features for each signal
    """

    # Initialize dictionary to store features for each signal
    features = []

    # Loop over each signal
    for i in range(2):
        # Perform wavelet packet decomposition
        coeffs = pywt.WaveletPacket(data=signals[i], wavelet=wavelet, mode='symmetric', maxlevel=level).get_level(level, 'freq')

        # Vectorize the coefficients
        coefficients = np.array([c.data for c in coeffs], dtype=object)

        # Store the coefficients as features for this signal
        features.append(coefficients.tolist())

    return features


In [269]:
dataset, scaler = getDataset()

100%|█████████████████████████████████████████████████████████████████████████████| 2241/2241 [00:09<00:00, 241.50it/s]


In [270]:
train_loader, test_loader = getSplit(dataset)

In [26]:
class RNNSingle(nn.Module):
    def __init__(self, input_dim=2, hidden_dim=256, num_layers=3, output_dim=6):
        super(RNNSingle, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_dim, hidden_dim, num_layers, batch_first=True)
        
        self.fc = nn.Sequential(
            nn.Linear(45056, 1024),
            nn.Dropout(0.5),
            nn.ReLU(),
            nn.Linear(1024, 256),
            nn.Dropout(0.5),
            nn.ReLU(),
            nn.Linear(256, 64),
            nn.Dropout(0.5),
            nn.ReLU(),
            nn.Linear(64, 6)
        )


    def forward(self, x):
        batch_size = x.shape[0]
        x = x.reshape(batch_size, -1, 2)
        
        h0 = torch.zeros(3, batch_size, 256).to(x.device)
        x, _ = self.rnn(x, h0)
        x = x.reshape(batch_size, -1)
        
        x = self.fc(x)
        
        return x


In [ ]:
class LSTM(nn.Module):
    def __init__(self, input_dim=2, hidden_dim=256, num_layers=1, output_dim=6):
        super(LSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        
        self.fc = nn.Sequential(
            nn.Linear(hidden_dim, 1024),
            nn.ReLU(),
            nn.Linear(1024, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(64, 6)
        
        )

    def forward(self, x):
        batch_size = x.shape[0]
        x = x.reshape(batch_size, -1, 2)
        # Initialize hidden state with zeros
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()
        # Initialize cell state with zeros
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()
        # Move tensors to GPU if available
        if torch.cuda.is_available():
            h0 = h0.cuda()
            c0 = c0.cuda()
            x = x.cuda()
        # Forward propagate LSTM
        out, _ = self.lstm(x)
        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        return out


In [294]:
class SiameseRNN(nn.Module):
    def __init__(self, input_size=22, hidden_size=10):
        super(SiameseRNN, self).__init__()
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size=input_size, hidden_size=hidden_size, num_layers=1, batch_first=True)
        self.fc = nn.Sequential(
            nn.Linear(hidden_size * 2, 6),

        )
    
    def forward(self, x):
        batch_size = x.shape[0]
        x1 = x[:, 0, :, :]
        x2 = x[:, 0, :, :]
        
        h0 = torch.zeros(1, batch_size, self.hidden_size).to(DEVICE)
        _, h1 = self.rnn(x1, h0)  # Add a batch dimension
        _, h2 = self.rnn(x2, h0)  # Add a batch dimension
        
        out = torch.cat([h1[-1], h2[-1]], dim=1)
        out = out.reshape(batch_size, -1)
        out = self.fc(out)
        return out

In [295]:
dataset[0]['input'].shape

torch.Size([2, 8, 22])

In [302]:
sRNN = SiameseRNN()

In [303]:
sRNN_train_loss, sRNN_val_loss =  train(train_loader, sRNN, val_loader=test_loader, LR=0.01, epochs=400)

Using: cuda:0


Epoch 136/400: 100%|████████████████████████████████| 56/56 [00:00<00:00, 214.13batch/s, lastLoss=0.122, valLoss=0.127]


Epoch 272/400: 100%|████████████████████████████████| 56/56 [00:00<00:00, 224.95batch/s, lastLoss=0.116, valLoss=0.116]


Epoch 400/400: 100%|████████████████████████████████| 56/56 [00:00<00:00, 213.74batch/s, lastLoss=0.115, valLoss=0.111]


Average train loss: 0.12480014465749263
Average validation loss: 0.1234719197625915


In [34]:
rnnSingle = RNNSingle()

In [35]:
rnnSingle_train_loss, rnnSingle_val_loss =  train(train_loader, rnnSingle, val_loader=test_loader, LR=0.0001, epochs=400)

Using: cuda:0


Epoch 3/400:  27%|█████████▍                         | 15/56 [00:00<00:00, 41.03batch/s, lastLoss=0.231, valLoss=0.187]


KeyboardInterrupt: 

In [36]:
lstm = LSTM()

In [37]:
lstm_train_loss, lstm_val_loss =  train(train_loader, lstm, val_loader=test_loader, LR=0.0001, epochs=400)

Using: cuda:0


Epoch 52/400:  75%|█████████████████████████▌        | 42/56 [00:02<00:00, 20.98batch/s, lastLoss=0.191, valLoss=0.183]


KeyboardInterrupt: 

In [38]:
import gc
torch.cuda.empty_cache()
gc.collect()

6772

# Try capturing per level

In [310]:
def waveletExtraction(signals, wavelet='db4', levels=3):
    """
    Extract wavelet packet features for each signal in the input array.

    Args:
    - signals (numpy.ndarray): a 2D array containing two 1D signals
    - wavelet (str): the name of the wavelet to use for decomposition
    - level (int): the level of wavelet packet decomposition to use

    Returns:
    - features (dict): a dictionary containing the wavelet packet features for each signal
    """

    # Initialize dictionary to store features for each signal
    features = []

    # Loop over each signal
    for i in range(2):
        # Perform wavelet packet decomposition
        wp = pywt.WaveletPacket(data=signals[i], wavelet=wavelet, mode='symmetric', maxlevel=levels)
        
        coeffs = []
        for level in range(1, levels+1):
            level_coeffs = np.array([n.data for n in wp.get_level(level, 'freq')])  # get the coefficients at the current level
            
            stats = np.array([
                [np.mean(level_coeffs[i]), np.std(level_coeffs[i]), np.ptp(level_coeffs[i])]
                 for i in range(len(level_coeffs))
                ])

            for stat in stats:
                for s in stat:
                    coeffs.append(s)

        # Stack the coefficients vertically to create the final array
        features.append(coeffs)
    return features


In [311]:
dataset, scaler = getDataset()

100%|█████████████████████████████████████████████████████████████████████████████| 2241/2241 [00:09<00:00, 241.53it/s]


In [312]:
train_loader, test_loader = getSplit(dataset)

In [115]:
class RNNSingle(nn.Module):
    def __init__(self, input_dim=2, hidden_dim=2048, num_layers=2, output_dim=6):
        super(RNNSingle, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_dim, hidden_dim, num_layers, batch_first=True)
        
        self.fc = nn.Sequential(
            nn.Linear(86016, 1024),
            nn.Dropout(0.5),
            nn.ReLU(),
            nn.Linear(1024, 256),
            nn.Dropout(0.5),
            nn.ReLU(),
            nn.Linear(256, 64),
            nn.Dropout(0.5),
            nn.ReLU(),
            nn.Linear(64, 6)
        )


    def forward(self, x):
        batch_size = x.shape[0]
        x = x.reshape(batch_size, -1, 2)
        
        h0 = torch.zeros(2, batch_size, self.hidden_dim).to(x.device)
        x, _ = self.rnn(x, h0)
        x = x.reshape(batch_size, -1)
        
        x = self.fc(x)
        
        return x


In [143]:
class LSTM(nn.Module):
    def __init__(self, input_dim=2, hidden_dim=512, num_layers=5, output_dim=6):
        super(LSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        
        self.fc = nn.Sequential(
            nn.Linear(hidden_dim, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 6)
        
        )

    def forward(self, x):
        batch_size = x.shape[0]
        x = x.reshape(batch_size, -1, 2)
        # Initialize hidden state with zeros
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()
        # Initialize cell state with zeros
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()
        # Move tensors to GPU if available
        if torch.cuda.is_available():
            h0 = h0.cuda()
            c0 = c0.cuda()
            x = x.cuda()
        # Forward propagate LSTM
        out, _ = self.lstm(x)
        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        return out


In [328]:
class SiameseRNN(nn.Module):
    def __init__(self, input_size=42, hidden_size=10):
        super(SiameseRNN, self).__init__()
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size=input_size, hidden_size=hidden_size, num_layers=1, batch_first=True)
        self.fc = nn.Sequential(
            nn.Linear(hidden_size * 2, 6),

        )
    
    def forward(self, x):
        batch_size = x.shape[0]
        
        x1 = x[:, 0, :].unsqueeze(1)
        x2 = x[:, 0, :].unsqueeze(1)
        h0 = torch.zeros(1, batch_size, self.hidden_size).to(DEVICE)
        _, h1 = self.rnn(x1, h0)  # Add a batch dimension
        _, h2 = self.rnn(x2, h0)  # Add a batch dimension
        
        out = torch.cat([h1[-1], h2[-1]], dim=1)
        out = out.reshape(batch_size, -1)
        out = self.fc(out)
        return out

In [329]:
sRNN = SiameseRNN()

In [330]:
sRNN_train_loss, sRNN_val_loss =  train(train_loader, sRNN, val_loader=test_loader, LR=0.01, epochs=400)

Using: cuda:0


Epoch 136/400: 100%|████████████████████████████████| 56/56 [00:00<00:00, 224.45batch/s, lastLoss=0.142, valLoss=0.137]


Epoch 146/400:  57%|██████████████████▎             | 32/56 [00:00<00:00, 207.12batch/s, lastLoss=0.139, valLoss=0.139]


KeyboardInterrupt: 

In [119]:
rnnSingle = RNNSingle()

In [120]:
rnnSingle_train_loss, rnnSingle_val_loss =  train(train_loader, rnnSingle, val_loader=test_loader, LR=0.0001, epochs=400)

Using: cuda:0


Epoch 81/400:  68%|███████████████████████▊           | 38/56 [00:01<00:00, 20.95batch/s, lastLoss=0.19, valLoss=0.185]


KeyboardInterrupt: 

In [144]:
lstm = LSTM()

In [145]:
lstm_train_loss, lstm_val_loss =  train(train_loader, lstm, val_loader=test_loader, LR=0.0001, epochs=400)

Using: cuda:0


Epoch 57/400:  36%|████████████▌                      | 20/56 [00:00<00:00, 39.41batch/s, lastLoss=0.188, valLoss=0.18]


KeyboardInterrupt: 

# Try not flattened stats

In [331]:
def waveletExtraction(signals, wavelet='db4', levels=3):
    """
    Extract wavelet packet features for each signal in the input array.

    Args:
    - signals (numpy.ndarray): a 2D array containing two 1D signals
    - wavelet (str): the name of the wavelet to use for decomposition
    - level (int): the level of wavelet packet decomposition to use

    Returns:
    - features (dict): a dictionary containing the wavelet packet features for each signal
    """

    # Initialize dictionary to store features for each signal
    features = []

    # Loop over each signal
    for i in range(2):
        # Perform wavelet packet decomposition
        wp = pywt.WaveletPacket(data=signals[i], wavelet=wavelet, mode='symmetric', maxlevel=levels)
        
        coeffs = []
        for level in range(1, levels+1):
            level_coeffs = np.array([n.data for n in wp.get_level(level, 'freq')])  # get the coefficients at the current level
            
            stats = np.array([
                [np.mean(level_coeffs[i]), np.std(level_coeffs[i]), np.ptp(level_coeffs[i])]
                 for i in range(len(level_coeffs))
                ])
            for i in range(len(level_coeffs)):
                coeffs.append([np.mean(level_coeffs[i]), np.std(level_coeffs[i]), np.ptp(level_coeffs[i])])

        # Stack the coefficients vertically to create the final array
        features.append(coeffs)
    return features


In [388]:
np.concatenate([[0, 1, 2], [0, 1, 2]])

array([0, 1, 2, 0, 1, 2])

In [395]:
def waveletExtraction(x, wavelet='db1', level=4):
# perform wavelet packet decomposition on signal 1
    wp = pywt.WaveletPacket(x[0], wavelet, mode='symmetric', maxlevel=level)
    coeffs1 = []
    for node in wp.get_level(level, 'natural'):
        if node.path.endswith('a') or node.path.endswith('d'):
            coeffs1.append(node.data)
    coeffs1 = np.concatenate(coeffs1)
    
    # perform wavelet packet decomposition on signal 2
    wp = pywt.WaveletPacket(x[1], wavelet, mode='symmetric', maxlevel=level)
    coeffs2 = []
    for node in wp.get_level(level, 'natural'):
        if node.path.endswith('a') or node.path.endswith('d'):
            coeffs2.append(node.data)
    coeffs2 = np.concatenate(coeffs2)
    
    # concatenate the two coefficient arrays
    feature_vector = np.concatenate((coeffs1, coeffs2))
    
    return feature_vector

In [398]:
dataset[0]['input'].shape

torch.Size([256])

In [399]:
dataset, scaler = getDataset()

100%|█████████████████████████████████████████████████████████████████████████████| 2241/2241 [00:08<00:00, 254.01it/s]


In [400]:
train_loader, test_loader = getSplit(dataset)

In [401]:
class RNNSingle(nn.Module):
    def __init__(self, input_dim=2, hidden_dim=512, num_layers=3, output_dim=6):
        super(RNNSingle, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.rnn1 = nn.RNN(input_dim, hidden_dim, num_layers, batch_first=True)
        self.rnn2 = nn.RNN(input_dim, hidden_dim, num_layers, batch_first=True)
        
        self.fc = nn.Sequential(
            nn.Linear(21504, 1024),
            nn.Dropout(0.5),
            nn.ReLU(),
            nn.Linear(1024, 256),
            nn.Dropout(0.5),
            nn.ReLU(),
            nn.Linear(256, 64),
            nn.Dropout(0.5),
            nn.ReLU(),
            nn.Linear(64, 6)
        )


    def forward(self, x):
        batch_size = x.shape[0]
        x = x.reshape(batch_size, -1, 2)
        
        h0 = torch.zeros(3, batch_size, self.hidden_dim).to(x.device)
        x, _ = self.rnn(x, h0)
        x = x.reshape(batch_size, -1)
        
        x = self.fc(x)
        
        return x


In [402]:
class LSTM(nn.Module):
    def __init__(self, input_dim=2, hidden_dim=256, num_layers=2, output_dim=6):
        super(LSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        
        self.fc = nn.Sequential(
            nn.Linear(hidden_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(32, 6)
        
        )

    def forward(self, x):
        batch_size = x.shape[0]
        x = x.reshape(batch_size, -1, 2)
        # Initialize hidden state with zeros
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()
        # Initialize cell state with zeros
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()
        # Move tensors to GPU if available
        if torch.cuda.is_available():
            h0 = h0.cuda()
            c0 = c0.cuda()
            x = x.cuda()
        # Forward propagate LSTM
        out, _ = self.lstm(x)
        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        return out


In [447]:
class SiameseRNN(nn.Module):
    def __init__(self, input_size=256, hidden_size=1024):
        super(SiameseRNN, self).__init__()
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size=input_size, hidden_size=hidden_size, num_layers=1, batch_first=True)
                
        self.fc = nn.Sequential(
            nn.Linear(hidden_size, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 6)
        
        )
    
    def forward(self, x):
        batch_size = x.shape[0]
        x = x.reshape(batch_size, 1, -1)
        
        #x1 = x[:, 0, :, :].reshape(batch_size, 1, -1)
        #x2 = x[:, 0, :, :].reshape(batch_size, 1, -1)
        
        h0 = torch.zeros(1, batch_size, self.hidden_size).to(DEVICE)
        _, h1 = self.rnn(x, h0)  # Add a batch dimension
       # _, h2 = self.rnn(x2, h0)  # Add a batch dimension
        
       # out = torch.cat([h1[-1], h2[-1]], dim=1)
        out=h1[-1]
        out = out.reshape(batch_size, -1)
        out = self.fc(out)
        return out

In [448]:
sRNN = SiameseRNN()

In [449]:
sRNN_train_loss, sRNN_val_loss =  train(train_loader, sRNN, val_loader=test_loader, LR=0.001, epochs=400)

Using: cuda:0


Epoch 136/400: 100%|██████████████████████████████| 56/56 [00:00<00:00, 205.12batch/s, lastLoss=0.0834, valLoss=0.0846]


Epoch 272/400: 100%|██████████████████████████████| 56/56 [00:00<00:00, 188.23batch/s, lastLoss=0.0802, valLoss=0.0845]


Epoch 400/400: 100%|██████████████████████████████| 56/56 [00:00<00:00, 168.67batch/s, lastLoss=0.0772, valLoss=0.0824]


Average train loss: 0.08700385513848491
Average validation loss: 0.09160339520840594


In [451]:
test(test_loader, sRNN, scaler)

100%|█████████████████████████████████████████████████████████████████████████████| 15/15 [00:00<00:00, 576.99 batch/s]


(86.84305953979492,
 array([93.70472 , 59.70045 , 99.71429 , 99.451385, 76.14395 , 92.34357 ],
       dtype=float32),
 0.08491914172967276)

In [175]:
rnnSingle = RNNSingle()

In [176]:
rnnSingle_train_loss, rnnSingle_val_loss =  train(train_loader, rnnSingle, val_loader=test_loader, LR=0.0001, epochs=400)

Using: cuda:0


Epoch 4/400:  16%|█████▉                               | 9/56 [00:00<00:00, 58.63batch/s, lastLoss=0.222, valLoss=0.19]


KeyboardInterrupt: 

In [186]:
lstm = LSTM()

In [187]:
lstm_train_loss, lstm_val_loss =  train(train_loader, lstm, val_loader=test_loader, LR=0.0001, epochs=400)

Using: cuda:0


Epoch 38/400:   0%|                                           | 0/56 [00:00<?, ?batch/s, lastLoss=0.206, valLoss=0.184]


KeyboardInterrupt: 

In [ ]:
def waveletExtraction(signal1, signal2, decomp_level=5, wavelet_func='db4'):
    """
    Perform wavelet packet decomposition on two 1D signals and return a matrix of features
    where each row corresponds to a single time step, and the features are ordered
    based on the wavelet packet decomposition coefficients.
    
    Parameters
    ----------
    signal1 : array_like
        1D array of samples for the first signal.
        
    signal2 : array_like
        1D array of samples for the second signal.
        
    decomp_level : int, optional
        Number of decomposition levels to use in the wavelet packet decomposition. Default is 5.
        
    wavelet_func : str, optional
        Name of the wavelet function to use in the wavelet packet decomposition. Default is 'db4'.
    
    Returns
    -------
    features : numpy.ndarray
        2D array of features extracted from the wavelet packet decomposition, where each row corresponds
        to a single time step and the columns represent the wavelet packet decomposition coefficients.
    """
    
    # perform wavelet packet decomposition on each signal
    wp1 = pywt.WaveletPacket(signal1, wavelet_func, mode='symmetric', maxlevel=decomp_level)
    wp2 = pywt.WaveletPacket(signal2, wavelet_func, mode='symmetric', maxlevel=decomp_level)
    
    # collect the wavelet packet decomposition coefficients for each signal
    coeffs1 = []
    coeffs2 = []
    for level in range(decomp_level+1):
        for node in wp1.get_level(level, 'natural'):
            coeffs1.append(node.data)
        for node in wp2.get_level(level, 'natural'):
            coeffs2.append(node.data)
            
    # concatenate the coefficients across time
    coeffs_concat = np.concatenate([coeffs1, coeffs2], axis=0)

    # reshape the coefficients back into a time series with multiple features
    num_features = len(coeffs_concat)
    num_samples = len(signal1)
    features = np.zeros((num_samples, num_features))
    for i in range(num_samples):
        start_idx = i*num_features
        end_idx = (i+1)*num_features
        features[i,:] = coeffs_concat[start_idx:end_idx]
        
    return features
